In [1]:
%pip install pyfftw scipy

/bin/bash: /home/co21btech11008/miniconda3/envs/cuNu/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import cunumeric as cu
import matplotlib.pyplot as plt
import pyfftw
from scipy.fft import fft2 as scipy_fft2
from legate.timing import time

def spinodal_pyfftw(n):
    Nx = Ny = n
    c = 0.5 * np.ones([Nx, Ny])
    np.random.seed(1024)
    random_num = np.random.normal(0, 0.01, (Nx, Ny))
    c = c - random_num

    dx = 1.0
    dy = 1.0
    dt = 0.5
    cnew = c

    delkx = 2 * np.pi / (Nx * dx)
    delky = 2 * np.pi / (Ny * dy)

    A = 1
    M = 1
    kappa = 1

    start_time = time()

    i_indices = np.arange(Nx)
    j_indices = np.arange(Ny)

    kx = np.where(i_indices <= Nx/2, i_indices * delkx, (i_indices - Nx) * delkx)
    ky = np.where(j_indices <= Ny/2, j_indices * delky, (j_indices - Ny) * delky)

    k2 = kx[:, np.newaxis]**2 + ky[np.newaxis, :]**2
    k4 = k2**2

    for m in range(50):
        for n in range(100):
            mult = (1 - cnew) * (1 - 2 * cnew)
            g = 2 * A * cnew * mult
            ghat = pyfftw.interfaces.numpy_fft.fft2(g)
            chat = pyfftw.interfaces.numpy_fft.fft2(cnew)
            chat = (chat - M * dt * k2 * ghat) / (1 + 2 * M * kappa * k4 * dt)
            cnew = pyfftw.interfaces.numpy_fft.ifft2(chat).real
            c = cnew

    end_time = time()
    execution_time = end_time - start_time
    return execution_time/1000

def spinodal_scipy(n):
    Nx = Ny = n
    c = 0.5 * np.ones([Nx, Ny])
    np.random.seed(1024)
    random_num = np.random.normal(0, 0.01, (Nx, Ny))
    c = c - random_num

    dx = 1.0
    dy = 1.0
    dt = 0.5
    cnew = c

    delkx = 2 * np.pi / (Nx * dx)
    delky = 2 * np.pi / (Ny * dy)

    A = 1
    M = 1
    kappa = 1

    start_time = time()

    i_indices = np.arange(Nx)
    j_indices = np.arange(Ny)

    kx = np.where(i_indices <= Nx/2, i_indices * delkx, (i_indices - Nx) * delkx)
    ky = np.where(j_indices <= Ny/2, j_indices * delky, (j_indices - Ny) * delky)

    k2 = kx[:, np.newaxis]**2 + ky[np.newaxis, :]**2
    k4 = k2**2

    for m in range(50):
        for n in range(100):
            mult = (1 - cnew) * (1 - 2 * cnew)
            g = 2 * A * cnew * mult
            ghat = scipy_fft2(g)
            chat = scipy_fft2(cnew)
            chat = (chat - M * dt * k2 * ghat) / (1 + 2 * M * kappa * k4 * dt)
            cnew = np.fft.ifft2(chat).real
            c = cnew

    end_time = time()
    execution_time = end_time - start_time
    return execution_time/1000

def spinodal_cunumeric(n):
    Nx = Ny = n
    c = 0.5 * cu.ones([Nx, Ny])
    cu.random.seed(1024)
    random_num = cu.random.normal(0, 0.01, (Nx, Ny))
    c = c - random_num

    dx = 1.0
    dy = 1.0
    dt = 0.5
    cnew = c

    delkx = 2 * np.pi / (Nx * dx)
    delky = 2 * np.pi / (Ny * dy)

    A = 1
    M = 1
    kappa = 1

    start_time = time()

    i_indices = cu.arange(Nx)
    j_indices = cu.arange(Ny)

    kx = cu.where(i_indices <= Nx/2, i_indices * delkx, (i_indices - Nx) * delkx)
    ky = cu.where(j_indices <= Ny/2, j_indices * delky, (j_indices - Ny) * delky)

    k2 = kx[:, cu.newaxis]**2 + ky[cu.newaxis, :]**2
    k4 = k2**2

    for m in range(50):
        for n in range(100):
            mult = (1 - cnew) * (1 - 2 * cnew)
            g = 2 * A * cnew * mult
            ghat = cu.fft.fft2(g)
            chat = cu.fft.fft2(cnew)
            chat = (chat - M * dt * k2 * ghat) / (1 + 2 * M * kappa * k4 * dt)
            cnew = cu.fft.ifft2(chat).real
            c = cnew

    end_time = time()
    execution_time = end_time - start_time
    return execution_time/1000

# Range of array sizes
sizes = [16, 32, 64, 128, 256, 512, 1024]
# sizes = [16, 32]

# Measure time taken for each size
time_pyfftw = []
time_scipy = []
time_cunumeric = []

for size in sizes:
    time_pyfftw.append(spinodal_pyfftw(size))
    time_scipy.append(spinodal_scipy(size))
    time_cunumeric.append(spinodal_cunumeric(size))
    print(f'size {size} done')

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(sizes, time_pyfftw, label='PyFFTW')
plt.plot(sizes, time_scipy, label='SciPy')
plt.plot(sizes, time_cunumeric, label='cunumeric')
plt.title('Spinodal 2D Performance Comparison')
plt.xlabel('Array Size')
plt.ylabel('Time (milliseconds)')
plt.legend()
plt.grid(True)
plt.show()


/home/co21btech11008/miniconda3/envs/cuNu/lib/python3.9/site-packages/legate/core/__init__.py:143: Warning: 
        Seeding the random number generator with a non-constant value 
        inside Legate can lead to undefined behavior and/or errors when 
        the program is executed with multiple ranks.
  warnings.warn(msg, Warning)
size 16 done
size 32 done
size 64 done
size 128 done
size 256 done
size 512 done


In [1]:
time_cuda = [0.007168, 0.008192, 0.009216, 0.012288, 0.029696, 0.082944, 0.316416] # in milliseconds

In [2]:
plt.figure(figsize=(10, 6))
plt.plot(sizes, time_pyfftw, label='PyFFTW')
plt.plot(sizes, time_scipy, label='SciPy')
plt.plot(sizes, time_cunumeric, label='cunumeric')
plt.plot(sizes,time_cuda, label='cuda')
plt.title('Spinodal 2D Performance Comparison')
plt.xlabel('Array Size')
plt.ylabel('Time (milliseconds)')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'plt' is not defined